In [1]:
cd ..

/afs/inf.ed.ac.uk/user/s13/s1320903/Neuroglycerin/neukrill-net-work


In [2]:
ls

calculate_normalisation_stats.py  predictions.pkl
check_test_score.py               pylearn2_cpu_model_function_builder.py
data/                             README.md
generate_hlf_cache.py             run_settings/
generate_prior_weighted_csv.py    settings.json
generate_prior_weighted_csv.py~   start_script.sh*
generate_prior_weighted_csv.pyc   test_attr.py
image_hack.sh*                    test_priors.py
LICENSE                           test.py*
logistic_sgd3.py                  test.py~
misc/                             train_attr.py
mlp3.py                           train_bow.py
mlp.py                            train.py*
mnist_train.py                    yaml_templates/
model.pkl                         y_pickle
notebooks/


In [5]:
import pylearn2.utils
import pylearn2.config
import theano
import neukrill_net.utils
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import holoviews as hl
%load_ext holoviews.ipython
import sklearn.metrics
import pickle

Welcome to the HoloViews IPython extension! (http://ioam.github.io/holoviews/)
Available magics: %compositor, %opts, %params, %view, %%labels, %%opts, %%view


:0: FutureWarning: IPython widgets are experimental and may change in the future.


Model with col_norms set to a higher value and augmentations turned on. We saved its pickle with predictions across all superclass vectors.

In [6]:
with open("predictions.pkl", "rb") as f:
    predictions = pickle.load(f)

Get first parent predictions out.

In [16]:
superclasses = predictions[:,121:(121+38)]
s = np.sum(superclasses, axis = 1)
any((s < 0.9999 )*(s>1.00001))

False

In [84]:
import neukrill_net.utils as utils
settings = neukrill_net.utils.Settings("settings.json")
import neukrill_net.encoding as enc
hier = enc.get_hierarchy(settings)

Get those superclasses that only have classes as children.

In [150]:
import neukrill_net.taxonomy as t
only_leaf_children = []

layer = t.TaxonomyLayer(1)

for s in hier[1]:
    flag = True
    for key, values in t.superclasses.items():
        for i, v in enumerate(values):
            if v == s and i != 1:
                flag = False
    if flag:
        if s not in only_leaf_children:
            only_leaf_children.append(s)

In [154]:
only_leaf_children

['acantharia',
 'appendicularians',
 'calanoid',
 'chaetognaths',
 'cydippid',
 'decapods_all',
 'detritus',
 'diatoms',
 'euphausiids_all_ages',
 'fish',
 'no_class',
 'oithona',
 'other_hydromedusae',
 'physonect',
 'pluteus',
 'pteropods',
 'radiolarian',
 'rocketship',
 'seastar',
 'sphaeronectes',
 'sub_hydromedusae1',
 'sub_hydromedusae2',
 'sub_protists',
 'trichodesmium',
 'tunicate',
 'unknown']

Create a list with keys being indices of a superclass (first parent) and values being all of the classes in it. 

In [167]:
import neukrill_net.taxonomy as t
layer = t.TaxonomyLayer(1)

backmap = {}
for i, c in enumerate(settings.classes):
    j = int(np.where(np.array(hier[1]) == layer[c])[0])
    if hier[1][j] in only_leaf_children:
        try:
            backmap[j] += [i]
        except:
            backmap[j] = [i]

In [168]:
backmap

{0: [0, 1, 2],
 1: [4, 5, 6, 7],
 2: [14, 15, 16, 17, 18, 19, 20, 21, 22, 26],
 4: [10, 11, 12],
 8: [29, 30],
 9: [32, 92, 94, 95],
 10: [33, 34, 35, 49],
 11: [36, 37],
 13: [47, 48],
 14: [50, 51, 52, 53, 54, 55],
 17: [8, 9],
 18: [24, 25],
 19: [58, 59, 66, 67, 68, 69, 70, 71, 74, 75, 76, 77],
 21: [104, 105],
 23: [38, 39, 40, 41, 45],
 25: [87, 88, 89],
 26: [90, 91],
 27: [97, 98],
 28: [42, 43],
 31: [99, 100, 101],
 32: [57, 60, 61, 62],
 33: [63, 64, 65, 72, 73],
 34: [82, 83, 85, 86],
 35: [108, 109, 110, 111],
 36: [113, 114, 115, 116, 117],
 37: [118, 119, 120]}

In [175]:
class_predictions = predictions[:, 0:121]
nr = class_predictions

Make a large array for class predictions for all images. Replace those predictions that correspond to only_leaf_children superclasses with its prediction weighted by the class prior. For other classes, use their own predictions.

In [176]:
for index,r in enumerate(superclasses):
    if index%1000 == 0:
        print index
    for i,j in enumerate(r):
        priors = []
        if i in backmap:
            d = sum([settings.class_priors[k] for k in backmap[i]])
            for a in backmap[i]:
                priors.append(settings.class_priors[a]/d)
            nr[index,backmap[i]] = j*np.array(priors)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000


Write a .csv file.

In [71]:
import os
os.path.basename(settings.image_fnames['test'][0])

u'1.jpg'

In [73]:
names = [os.path.basename(n) for n in settings.image_fnames['test']]

[u'1.jpg',
 u'10.jpg',
 u'100.jpg',
 u'1000.jpg',
 u'10000.jpg',
 u'100000.jpg',
 u'100001.jpg',
 u'100002.jpg',
 u'100003.jpg',
 u'100004.jpg',
 u'100005.jpg',
 u'100006.jpg',
 u'100007.jpg',
 u'100008.jpg',
 u'10001.jpg',
 u'100011.jpg',
 u'100012.jpg',
 u'100013.jpg',
 u'100014.jpg',
 u'100015.jpg',
 u'100016.jpg',
 u'100017.jpg',
 u'100018.jpg',
 u'100019.jpg',
 u'10002.jpg',
 u'100020.jpg',
 u'100021.jpg',
 u'100022.jpg',
 u'100023.jpg',
 u'100024.jpg',
 u'100025.jpg',
 u'100026.jpg',
 u'100027.jpg',
 u'100028.jpg',
 u'10003.jpg',
 u'100030.jpg',
 u'100031.jpg',
 u'100032.jpg',
 u'100033.jpg',
 u'100034.jpg',
 u'100035.jpg',
 u'100036.jpg',
 u'100037.jpg',
 u'100038.jpg',
 u'100039.jpg',
 u'10004.jpg',
 u'100041.jpg',
 u'100042.jpg',
 u'100043.jpg',
 u'100045.jpg',
 u'100046.jpg',
 u'100047.jpg',
 u'100048.jpg',
 u'100049.jpg',
 u'10005.jpg',
 u'100050.jpg',
 u'100051.jpg',
 u'100052.jpg',
 u'100053.jpg',
 u'100054.jpg',
 u'100055.jpg',
 u'100057.jpg',
 u'100059.jpg',
 u'10006.jpg

In [74]:
import neukrill_net.utils as utils
utils.write_predictions('superclass_predictions.csv', nr, names, settings.classes)

In [77]:
utils.write_predictions('class_predictions.csv', class_predictions, names, settings.classes)